# Import Package and csv

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.pandas.set_option("display.max_columns", None)
# Create Dataframe
df = pd.read_csv(r"EasyVisa.csv")
# Print shape of dataset
print(df.shape)

(25480, 12)


## Data Cleaning

# Handling missing values
    Handle missing values
    Handle duplicate values
    check data type
    understand the dataset

Check null Values

In [3]:
# Features with NAN value
feature_with_na = [feature for feature in df.columns if(df[feature].isnull().sum()>=1)]

for feat in feature_with_na:
    print(feat,np.round(df[feat].isnull().sum()*100),"% missing values")

In [4]:
feature_with_na

[]

There is no feature with missing value

### Other Data Cleaning step

Handling duplicate value